In [2]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

# Creating Computer Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'your-compute-cluster'
try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as e:
        print(e)

In [3]:
# Datastores are references to storage locations such as Azure Storage blob containers. Every workspace comes with a default datastore. 

default_ds = ws.get_default_datastore()

# loop through all datastores and find which one is actually the default
for ds_name in ws.datastores:
    print(ds_name,'- Default =', ds_name == default_ds.name)

workspaceworkingdirectory - Default = False
workspacefilestore - Default = False
workspaceartifactstore - Default = False
workspaceblobstore - Default = True


In [4]:
datastore = ws.datastores['workspaceblobstore']

In [5]:
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-db2e5a15-d42a-49d7-b494-181af9d9e23d",
  "account_name": "mlwdp100labs1149010901",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [7]:
# Uploading data to Datastore
from azureml.core import Dataset
from azureml.data.datapath import DataPath

Dataset.File.upload_directory(src_dir='data',target=(datastore,'datas-diabetes'))

Validating arguments.
Arguments validated.
'overwrite' is set to False. Any file already present in the target will be skipped.'
Uploading files from '/Users/timothy/Programming/azure_training/module3/working_with_data/data' to 'datas-diabetes'
Copying 1 files with concurrency set to 1
Skipped /Users/timothy/Programming/azure_training/module3/working_with_data/data/test_diabetes.csv, file 1 out of 1. Target already exists.
Files copied=0, skipped=1, failed=0
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/datas-diabetes')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

In [5]:
import os
path = os.getcwd()
print(path)

/Users/timothy/Programming/azure_training/module3/working_with_data
